<a href="https://colab.research.google.com/github/AmoohM/Flower-CNN-Classifier/blob/master/FlowersCNNClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import required packages

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
import shutil
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Load Images

In [ ]:
_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
zip_file= tf.keras.utils.get_file(origin=_URL,fname="flower_photos.tgz" , extract= True)
base_dir= os.path.join(os.path.dirname(zip_file) , 'flower_photos')

Classes for the flowers

In [ ]:
classes = ['roses', 'daisy', 'dandelion', 'sunflowers', 'tulips']

Separate images into training and testing sets

In [ ]:
for cl in classes:
  img_path = os.path.join(base_dir, cl)
  images = glob.glob(img_path + '/*.jpg')
  print("{}: {} Images".format(cl, len(images)))
  num_train = int(round(len(images)*0.8))
  train, val = images[:num_train], images[num_train:]

  for t in train:
    if not os.path.exists(os.path.join(base_dir, 'train', cl)):
      os.makedirs(os.path.join(base_dir, 'train', cl))
    shutil.move(t, os.path.join(base_dir, 'train', cl))

  for v in val:
    if not os.path.exists(os.path.join(base_dir, 'val', cl)):
      os.makedirs(os.path.join(base_dir, 'val', cl))
    shutil.move(v, os.path.join(base_dir, 'val', cl))

train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

Data Generation and Augmentation

In [ ]:
IMAGE_SHAPE=150
BATCH_SIZE=100

image_gen_train = ImageDataGenerator(rescale = 1./255 ,horizontal_flip= True ,rotation_range= 50 ,zoom_range=0.6 ,width_shift_range=[-0.2,0.2] , height_shift_range=[-0.2,0.2] )
train_img_gen = image_gen_train.flow_from_directory(batch_size = BATCH_SIZE ,directory = train_dir ,shuffle=True ,target_size = (IMAGE_SHAPE,IMAGE_SHAPE) ,class_mode='sparse')

#Validation image generator
image_gen_validate= ImageDataGenerator(rescale=1./255)
val_img_gen= image_gen_validate.flow_from_directory(batch_size= BATCH_SIZE ,directory=val_dir ,target_size = (IMAGE_SHAPE,IMAGE_SHAPE) ,class_mode='sparse')

Create the CNN

In [ ]:
model = Sequential()

model.add(Conv2D(32, 3, padding='same', activation='relu', input_shape=(IMAGE_SHAPE,IMAGE_SHAPE, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))

model.add(Dropout(0.2))
model.add(Dense(5))

model.summary()

Compile the Model


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Train the Model

In [ ]:
epochs=80

history = model.fit(train_img_gen ,steps_per_epoch=int(np.ceil(train_img_gen.n / float(BATCH_SIZE))) ,epochs=epochs ,validation_data=val_img_gen ,validation_steps=int(np.ceil(val_img_gen.n / float(BATCH_SIZE))))


Plot validation and Training Graphs

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()